In [1]:
import fabric

fabric.__version__

'2.6.0'

# 2 使用fabric玩转远程服务器管理

## 2.1 连接到远程服务器并执行终端命令

In [2]:
from fabric import Connection

# 建议将ssh连接所需参数变量化
user = '用户名'
host = 'host地址'
password = '密码'


# 利用fabric.Connection快捷创建连接
c = Connection(host=f'{user}@{host}',
               connect_kwargs=dict(
                   password=password
               ))

# 利用run方法直接执行传入的命令
c.run('pwd');

/home/feffery


In [3]:
# hide=True抑制run()过程对执行结果的自动打印
output = c.run('df -h', hide=True).stdout
print(output)

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        1.8G     0  1.8G   0% /dev
tmpfs           1.8G     0  1.8G   0% /dev/shm
tmpfs           1.8G  1.3M  1.8G   1% /run
tmpfs           1.8G     0  1.8G   0% /sys/fs/cgroup
/dev/vda1        40G  4.6G   33G  13% /
tmpfs           365M     0  365M   0% /run/user/0
tmpfs           365M     0  365M   0% /run/user/1000



## 2.2 配置sudo命令密码自动预填入

### 2.2.1 方式1：配合invoke.Responder

In [4]:
from invoke import Responder

# 配置命令行内容监听规则
sudopass = Responder(
    pattern=f'\[sudo\] password for {user}:',
    response=password+'\n'
)

# 注意需要设置pty=True
c.run('sudo pwd', pty=True, watchers=[sudopass]);

[sudo] password for feffery: 
/home/feffery


### 2.2.2 方式2：利用fabric.Config设置sudo密码

In [5]:
from fabric import Config

# 预先配置sudo密码
config = Config(overrides={
    'sudo': {
        'password': password
    }
})

c = Connection(host=f'{user}@{host}', 
               connect_kwargs={'password': password},
               config=config)

c.sudo('pwd');

[sudo] password: 

/home/feffery


## 2.3 远程文件传输

### 2.3.1 从本地上传文件到服务器

In [6]:
c = Connection(host=f'{user}@{host}', 
               connect_kwargs={'password': password})

# 创建示例文件
with open('file_transfer.txt', 'w') as d:
    d.write('1')

# 利用put方法上传至服务器
c.put('file_transfer.txt', remote='/home/feffery/')

# 打印已上传文件内容
c.run('cat /home/feffery/file_transfer.txt');

1

### 2.3.2 从服务器下载指定文件到本地

In [7]:
c = Connection(host=f'{user}@{host}', 
               connect_kwargs={'password': password})

# 向文件末尾追加行
c.run('echo "\n2" >> file_transfer.txt')

<Result cmd='echo "\n2" >> file_transfer.txt' exited=0>

In [8]:
c.get('/home/feffery/file_transfer.txt')

In [9]:
print(open('file_transfer.txt').read())

1
2

